In [1]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
BRANCH = 'main'
#!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [2]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
from nemo.utils import logging

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

[NeMo W 2022-04-20 18:09:14 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-04-20 18:09:15 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


# Task Description
**Joint Intent and Slot classification** - is a task of classifying an Intent and detecting all relevant Slots (Entities)
for this Intent in a query.
For example, in the query:  `What is the weather in Santa Clara tomorrow morning?`, we would like to classify the query
as a `weather` Intent, and detect `Santa Clara` as a `location` slot and `tomorrow morning` as a `date_time` slot.
Intents and Slots names are usually task specific and defined as labels in the training data.
This is a fundamental step that is executed in any task-driven Conversational Assistant.

Our Bert based model implementation enables to train and then detect both of these tasks together.

**Multi Label Joint Intent and Slot classification** - is very similar to the task above, but instead of only classifying a single Intent, the task can predict multiple different intents for each query. For example, for the query `Yes, please tell me the weather`, we might want the intents for this utterance to be `yes` and `weather`. You can skip to that tutorial [here](#multi-label)


# Dataset and NeMo data format

In this tutorial we are going to use a virtual assistant interaction data set that can be downloaded from here: https://github.com/xliuhw/NLU-Evaluation-Data.
There are about 10K training and 1K testing queries which cover 64 various Intents and 55 Slots. 

To work with NeMo NLP classification model, this dataset should be first converted to the NeMo format, which requires next files:
- **dict.intents.csv** - list of all intent names in the data. One line per an intent name.
- **dict.slots.csv** - list of all slot names in the data. One line per a slot name. It is possible to use both: B- I- notations, for separating between first and intermediate tokens for multi token slots. Or just use one slot type for each token of multi token slot. Our recommendation is to use later one, since it is simpler and there is no visible degradation in performance.
- **train.tsv/test.tsv** - contain original queries, one per line, and intent number separated by tab. For example: `what alarms do i have set right now	0`. Intent numbers are according to the intent line in the intent dictionary file (dict.intents.csv) starting from 0. First line of these files contains a header line: `sentence \tab label`.
- **train_slot.tvs/test_slot.tsv** - contain one line per a query, where instead each token there is a number of the token from the slots dictionary file (dict.slots.csv), starting from 0. Last 'out-of scope' token is usually located in the last line of the dictionary. Example: `54 0 0 54 54 12 12` (numbers separated by space). No header line in these files.

NeMo provides **import_dataset.py** converter for few reference datasets (Assistant / Atis / Snips) which converts them to the NeMo data format for the Intent and Slot classification model. If you have your own annotated dataset in a different format, you will need to write a data converter. Possible recommended format for your own annotation, is to have one text file per all examples of one intent. With one line per query in a form like: `did i set an alarm to [alarm_type : wake up] in the [timeofday : morning]`, using brackets to define slot names. This is very similar to the assistant format from this example and you can use its converter to NeMo format with small changes. 

You can run this utility as follows:

**python examples/nlp/intent_slot_classification/data/import_datasets.py --dataset_name=assistant --source_data_dir=source_dir_name --target_data_dir=target_dir_name**


# Download, preprocess and explore the dataset
## Download the dataset and convert it to the NeMo format

In [3]:
# you can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "."
NEMO_DIR = '.'

# download the converter files from github for the purpose of this tutorial
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/import_datasets.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/assistant_utils.py', NEMO_DIR)

100% [............................................................] 5619 / 5619

'./assistant_utils.py'

In [4]:
# download and unzip the example dataset from github
print('Downloading dataset...')
wget.download('https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip', DATA_DIR)
! unzip {DATA_DIR}/NLU-Evaluation-Data-master.zip -d {DATA_DIR}

-1 / unknownArchive:  ./NLU-Evaluation-Data-master.zip
f6071b496b17d71e6eb43f543af0707f4ff30557
   creating: ./NLU-Evaluation-Data-master/
   creating: ./NLU-Evaluation-Data-master/AnnotatedData/
  inflating: ./NLU-Evaluation-Data-master/AnnotatedData/NLU-Data-Home-Domain-Annotated-All.csv  
   creating: ./NLU-Evaluation-Data-master/AnnotationGuidelines/
  inflating: ./NLU-Evaluation-Data-master/AnnotationGuidelines/NLU-MLtraining-AnnotationSchma.docx  
   creating: ./NLU-Evaluation-Data-master/Collected-Original-Data/
  inflating: ./NLU-Evaluation-Data-master/Collected-Original-Data/paraphrases_and_intents_26k_normalised_all.csv  
   creating: ./NLU-Evaluation-Data-master/CrossValidation/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_confirm.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_dontcare.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_explain.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_joke.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_negate.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_praise.cs

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_3/trainset/general_repeat.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_3/trainset/iot_cleaning.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_3/trainset/iot_coffee.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_3/trainset/iot_hue_lightchange.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_3/trainset/iot_hue_lightdim.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_3/trainset/iot_hue_lightoff.csv  
 

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_6/trainset/calendar_query.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_6/trainset/calendar_remove.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_6/trainset/calendar_set.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_6/trainset/cooking_recipe.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_6/trainset/datetime_convert.csv  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_6/trainset/datetime_query.csv  
  i

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_9/testset/text/iot_wemo_on.txt  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_9/testset/text/lists_createoradd.txt  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_9/testset/text/lists_query.txt  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_9/testset/text/lists_remove.txt  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_9/testset/text/music_likeness.txt  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_9/testset/text/music_

...

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_4/merged/intents/takeaway_query.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_4/merged/intents/transport_query.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_4/merged/intents/transport_taxi.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_4/merged/intents/transport_ticket.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_4/merged/intents/transport_traffic.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_4

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_7/merged/intents/cooking_recipe.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_7/merged/intents/datetime_convert.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_7/merged/intents/datetime_query.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_7/merged/intents/email_addcontact.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_7/merged/intents/email_query.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_7/merg

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/transport_taxi.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/transport_ticket.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/transport_traffic.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/weather_query.json  
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/


  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/merged/RasaNlu_Merged_KFold_2_2018_03_22-13_01_25_169.json  
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/mergedTestset/
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/mergedTestset/RasaNluTestset_Merged_2018_03_22-13_01_25_169.json  
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/rasaTrainConfig/
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/rasaTrainConfig/configLatest_KFold_2_mitie_2018_03_22-13_01_25_169.json  
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_

  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_2/merged/intents/WatsonNluIntentsMerged_2018_03_22-13_01_25_169.csv  
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_3/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_3/merged/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_3/merged/entities/
  inflating: ./NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_3/merged/entities/watsonEntities_2018_03_22-13_01_25_169.csv  
   creating: ./NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_3/merged/intents/
  infla

In [5]:
# convert the dataset to the NeMo format
!python {NEMO_DIR}/import_datasets.py --dataset_name=assistant --source_data_dir={DATA_DIR}/NLU-Evaluation-Data-master --target_data_dir={DATA_DIR}/nemo_format


[NeMo W 2022-04-20 18:09:37 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-04-20 18:09:37 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo I 2022-04-20 18:09:38 assistant_utils:127] Processing assistant commands dataset and store at ./nemo_format
[NeMo I 2022-04-20 18:09:38 assistant_utils:31] Copying files to input folder: ./NLU-Evaluation-Data-master/dataset
Found 64 intents
Found 55 slot types


## Data exploration
You can see the dataset in both the original and NeMo's formats. We have here 65 different Intents and 55 Slots, which could be typical commands for virtual assistants. Out of scope slot has the name 'O' and is the last in the dictionary of Slots. And we can see examples of queries and also format of training intent and slot files. 

In [6]:
# list of queries divided by intent files in the original training dataset
! ls -l {DATA_DIR}/NLU-Evaluation-Data-master/dataset/trainset

total 1488
-rw-r--r-- 1 root root 22459 Mar  9 12:35 alarm_query.csv
-rw-r--r-- 1 root root 13045 Mar  9 12:35 alarm_remove.csv
-rw-r--r-- 1 root root 23552 Mar  9 12:35 alarm_set.csv
-rw-r--r-- 1 root root  7911 Mar  9 12:35 audio_volume_down.csv
-rw-r--r-- 1 root root 14525 Mar  9 12:35 audio_volume_mute.csv
-rw-r--r-- 1 root root 14732 Mar  9 12:35 audio_volume_up.csv
-rw-r--r-- 1 root root 27361 Mar  9 12:35 calendar_query.csv
-rw-r--r-- 1 root root 27068 Mar  9 12:35 calendar_remove.csv
-rw-r--r-- 1 root root 34492 Mar  9 12:35 calendar_set.csv
-rw-r--r-- 1 root root 25138 Mar  9 12:35 cooking_recipe.csv
-rw-r--r-- 1 root root 16357 Mar  9 12:35 datetime_convert.csv
-rw-r--r-- 1 root root 23226 Mar  9 12:35 datetime_query.csv
-rw-r--r-- 1 root root 12869 Mar  9 12:35 email_addcontact.csv
-rw-r--r-- 1 root root 23510 Mar  9 12:35 email_query.csv
-rw-r--r-- 1 root root 26902 Mar  9 12:35 email_querycontact.csv
-rw-r--r-- 1 root root 30069 Mar  9 12:35 email_sendemail.csv
-rw-r--r-- 

In [7]:
# print all intents from the NeMo format intent dictionary
!echo 'Intents: ' $(wc -l < {DATA_DIR}/nemo_format/dict.intents.csv)
! cat {DATA_DIR}/nemo_format/dict.intents.csv

Intents:  64
alarm_query
alarm_remove
alarm_set
audio_volume_down
audio_volume_mute
audio_volume_up
calendar_query
calendar_remove
calendar_set
cooking_recipe
datetime_convert
datetime_query
email_addcontact
email_query
email_querycontact
email_sendemail
general_affirm
general_commandstop
general_confirm
general_dontcare
general_explain
general_joke
general_negate
general_praise
general_quirky
general_repeat
iot_cleaning
iot_coffee
iot_hue_lightchange
iot_hue_lightdim
iot_hue_lightoff
iot_hue_lighton
iot_hue_lightup
iot_wemo_off
iot_wemo_on
lists_createoradd
lists_query
lists_remove
music_likeness
music_query
music_settings
news_query
play_audiobook
play_game
play_music
play_podcasts
play_radio
qa_currency
qa_definition
qa_factoid
qa_maths
qa_stock
recommendation_events
recommendation_locations
recommendation_movies
social_post
social_query
takeaway_order
takeaway_query
transport_query
transport_taxi
transport_ticket
transport_traffic
weather_query


In [8]:
# print all slots from the NeMo format slot dictionary
!echo 'Slots: ' $(wc -l < {DATA_DIR}/nemo_format/dict.slots.csv)
! cat {DATA_DIR}/nemo_format/dict.slots.csv

Slots:  55
alarm_type
app_name
artist_name
audiobook_author
audiobook_name
business_name
business_type
change_amount
coffee_type
color_type
cooking_type
currency_name
date
definition_word
device_type
drink_type
email_address
email_folder
event_name
food_type
game_name
game_type
general_frequency
house_place
ingredient
joke_type
list_name
meal_type
media_type
movie_name
movie_type
music_album
music_descriptor
music_genre
news_topic
order_type
person
personal_info
place_name
player_setting
playlist_name
podcast_descriptor
podcast_name
radio_name
relation
song_name
time
time_zone
timeofday
transport_agency
transport_descriptor
transport_name
transport_type
weather_descriptor
O


In [9]:
# examples from the intent training file
! head -n 10 {DATA_DIR}/nemo_format/train.tsv

sentence	label
what alarms do i have set right now	0
checkout today alarm of meeting	0
report alarm settings	0
see see for me the alarms that you have set tomorrow morning	0
is there an alarm for ten am	0
confirm the alarm time	0
show my alarms	0
at what time have you set alarm for me	0
please list active alarms	0


In [10]:
# examples from the slot training file
! head -n 10 {DATA_DIR}/nemo_format/train_slots.tsv

54 54 54 54 54 54 54 54
54 12 54 54 54
54 54 54
54 54 54 54 54 54 54 54 54 54 12 48
54 54 54 54 54 46 46
54 54 54 54
54 54 54
54 54 54 54 54 54 54 54 54
54 54 54 54
54 54 54 54 54 54


# Training model

## Model configuration

Our Joint Intent and Slot classification model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model with an Intent and Slot Classification layer on top of it.

All model and training parameters are defined in the **intent_slot_classification_config.yaml** config file. This file is located in the folder **examples/nlp/intent_slot_classification/conf/**. It contains 2 main sections:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

We will download the config file from repository for the purpose of the tutorial. If you have a version of NeMo installed locally, you can use it from the above folder.

In [ ]:
# download the model config file from repository for the purpose of this example
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/conf/intent_slot_classification_config.yaml', NEMO_DIR)

# print content of the config file
config_file = "intent_slot_classification_config.yaml"
print(config_file)
config = OmegaConf.load(config_file)
print(OmegaConf.to_yaml(config))

## Setting up Data within the config

Among other things, the config file contains dictionaries called train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

The converter utility creates both training and evaluation files in the same directory, so we need to specify `model.data_dir` parameter to this directory. Also notice that some config lines, including `model.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user.

`config.model.intent_loss_weight` parameter - is a balance of training loss between Intent and Slot losses, a number between 0 to 1. Its default value is 0.6 which gives slightly higher priority to the Intent loss and it empirically works quite well. You can experiment with this value if you like.
Also you can try to change `config.model.class_balancing` parameter to `weighted_loss` and see if you get better accuracy.

Let's now add the data directory path to the config.

In [ ]:
config.model.data_dir = f'{DATA_DIR}/nemo_format'

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem. `config.trainer.max_epochs` - param defines number of training epochs. Usually 50-100 epochs or less should be enough to train on your data. Let's instantiate the Trainer object.

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator

config.trainer.precision = 16 if torch.cuda.is_available() else 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = O1

# remove distributed training flags
config.trainer.strategy = None

# setup a small number of epochs for demonstration purposes of this tutorial
config.trainer.max_epochs = 5

trainer = pl.Trainer(**config.trainer)

## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it. Model check points during training will be saved in this directory. 

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
# the exp_dir provides a path to the current experiment for easy access
print(str(exp_dir))

## Initializing the model and Training

Initial statistics of the dataset will be displayed at the beginning of the training and then Intent and Slot classification report will be displayed after each training epoch.

In [ ]:
# initialize the model
model = nemo_nlp.models.IntentSlotClassificationModel(config.model, trainer=trainer)

# train
trainer.fit(model)

After training for 5 epochs, which should take no more than few minutes, you can expect training precision for this data set to be around these numbers (the accuracy will gradually continue to improve for this dataset up to about 50 epochs of training): 
```
Intents:
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                               94.74      94.74      94.74         19
    alarm_remove (label_id: 1)                             100.00     100.00     100.00         11
    alarm_set (label_id: 2)                                 85.71      94.74      90.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                        100.00      86.67      92.86         15
    audio_volume_up (label_id: 5)                           56.52     100.00      72.22         13
    calendar_query (label_id: 6)                            55.00      57.89      56.41         19
    calendar_remove (label_id: 7)                           88.89      84.21      86.49         19
    calendar_set (label_id: 8)                              81.25      68.42      74.29         19
    cooking_recipe (label_id: 9)                            86.36     100.00      92.68         19
    datetime_convert (label_id: 10)                          0.00       0.00       0.00          8
    datetime_query (label_id: 11)                           65.52     100.00      79.17         19
    email_addcontact (label_id: 12)                        100.00      12.50      22.22          8
    email_query (label_id: 13)                              83.33      78.95      81.08         19
    email_querycontact (label_id: 14)                       62.50      78.95      69.77         19
    email_sendemail (label_id: 15)                          70.83      89.47      79.07         19
    general_affirm (label_id: 16)                           95.00     100.00      97.44         19
    general_commandstop (label_id: 17)                     100.00     100.00     100.00         19
    general_confirm (label_id: 18)                         100.00     100.00     100.00         19
    general_dontcare (label_id: 19)                        100.00     100.00     100.00         19
    general_explain (label_id: 20)                         100.00      94.74      97.30         19
    general_joke (label_id: 21)                            100.00     100.00     100.00         12
    general_negate (label_id: 22)                           95.00     100.00      97.44         19
    general_praise (label_id: 23)                          100.00      94.74      97.30         19
    general_quirky (label_id: 24)                           40.00      10.53      16.67         19
    general_repeat (label_id: 25)                          100.00     100.00     100.00         19
    iot_cleaning (label_id: 26)                             84.21     100.00      91.43         16
    iot_coffee (label_id: 27)                               94.74      94.74      94.74         19
    iot_hue_lightchange (label_id: 28)                      94.44      89.47      91.89         19
    iot_hue_lightdim (label_id: 29)                        100.00      83.33      90.91         12
    iot_hue_lightoff (label_id: 30)                         89.47      89.47      89.47         19
    iot_hue_lighton (label_id: 31)                           0.00       0.00       0.00          3
    iot_hue_lightup (label_id: 32)                          81.25      92.86      86.67         14
    iot_wemo_off (label_id: 33)                             60.00     100.00      75.00          9
    iot_wemo_on (label_id: 34)                             100.00      14.29      25.00          7
    lists_createoradd (label_id: 35)                        78.95      78.95      78.95         19
    lists_query (label_id: 36)                              78.95      78.95      78.95         19
    lists_remove (label_id: 37)                             90.00      94.74      92.31         19
    music_likeness (label_id: 38)                           70.59      66.67      68.57         18
    music_query (label_id: 39)                              77.78      73.68      75.68         19
    music_settings (label_id: 40)                            0.00       0.00       0.00          7
    news_query (label_id: 41)                               77.78      73.68      75.68         19
    play_audiobook (label_id: 42)                           90.00      94.74      92.31         19
    play_game (label_id: 43)                                80.00      84.21      82.05         19
    play_music (label_id: 44)                               53.85      73.68      62.22         19
    play_podcasts (label_id: 45)                            89.47      89.47      89.47         19
    play_radio (label_id: 46)                               93.75      78.95      85.71         19
    qa_currency (label_id: 47)                              95.00     100.00      97.44         19
    qa_definition (label_id: 48)                            85.00      89.47      87.18         19
    qa_factoid (label_id: 49)                               45.16      73.68      56.00         19
    qa_maths (label_id: 50)                                100.00     100.00     100.00         14
    qa_stock (label_id: 51)                                 95.00     100.00      97.44         19
    recommendation_events (label_id: 52)                    94.44      89.47      91.89         19
    recommendation_locations (label_id: 53)                 94.74      94.74      94.74         19
    recommendation_movies (label_id: 54)                   100.00     100.00     100.00         10
    social_post (label_id: 55)                              90.00      94.74      92.31         19
    social_query (label_id: 56)                             94.74     100.00      97.30         18
    takeaway_order (label_id: 57)                           93.75      78.95      85.71         19
    takeaway_query (label_id: 58)                           85.71      94.74      90.00         19
    transport_query (label_id: 59)                          83.33      78.95      81.08         19
    transport_taxi (label_id: 60)                          100.00     100.00     100.00         18
    transport_ticket (label_id: 61)                         89.47      89.47      89.47         19
    transport_traffic (label_id: 62)                       100.00     100.00     100.00         19
    weather_query (label_id: 63)                           100.00      89.47      94.44         19
    -------------------
    micro avg                                               85.04      85.04      85.04       1076
    macro avg                                               81.13      80.81      79.36       1076
    weighted avg                                            84.10      85.04      83.54       1076
    
Slots:
    label                                                precision    recall       f1           support   
    alarm_type (label_id: 0)                                 0.00       0.00       0.00          0
    app_name (label_id: 1)                                   0.00       0.00       0.00          6
    artist_name (label_id: 2)                                0.00       0.00       0.00         21
    audiobook_author (label_id: 3)                           0.00       0.00       0.00          1
    audiobook_name (label_id: 4)                             0.00       0.00       0.00         18
    business_name (label_id: 5)                             60.00      56.60      58.25         53
    business_type (label_id: 6)                              0.00       0.00       0.00         24
    change_amount (label_id: 7)                              0.00       0.00       0.00         25
    coffee_type (label_id: 8)                                0.00       0.00       0.00          4
    color_type (label_id: 9)                                 0.00       0.00       0.00         12
    cooking_type (label_id: 10)                              0.00       0.00       0.00          0
    currency_name (label_id: 11)                            84.09      75.51      79.57         49
    date (label_id: 12)                                     57.95      91.07      70.83        112
    definition_word (label_id: 13)                           0.00       0.00       0.00         20
    device_type (label_id: 14)                              74.55      51.25      60.74         80
    drink_type (label_id: 15)                                0.00       0.00       0.00          0
    email_address (label_id: 16)                             0.00       0.00       0.00         14
    email_folder (label_id: 17)                              0.00       0.00       0.00          1
    event_name (label_id: 18)                              100.00      13.24      23.38         68
    food_type (label_id: 19)                                51.72      69.77      59.41         43
    game_name (label_id: 20)                                60.00      14.29      23.08         21
    game_type (label_id: 21)                                 0.00       0.00       0.00          0
    general_frequency (label_id: 22)                         0.00       0.00       0.00          9
    house_place (label_id: 23)                              93.33      42.42      58.33         33
    ingredient (label_id: 24)                                0.00       0.00       0.00          6
    joke_type (label_id: 25)                                 0.00       0.00       0.00          4
    list_name (label_id: 26)                                 0.00       0.00       0.00         21
    meal_type (label_id: 27)                                 0.00       0.00       0.00          0
    media_type (label_id: 28)                                0.00       0.00       0.00         37
    movie_name (label_id: 29)                                0.00       0.00       0.00          0
    movie_type (label_id: 30)                                0.00       0.00       0.00          0
    music_album (label_id: 31)                               0.00       0.00       0.00          0
    music_descriptor (label_id: 32)                          0.00       0.00       0.00          3
    music_genre (label_id: 33)                               0.00       0.00       0.00          9
    news_topic (label_id: 34)                                0.00       0.00       0.00         17
    order_type (label_id: 35)                                0.00       0.00       0.00         17
    person (label_id: 36)                                   44.86      92.31      60.38         52
    personal_info (label_id: 37)                             0.00       0.00       0.00         20
    place_name (label_id: 38)                               71.25      77.03      74.03        148
    player_setting (label_id: 39)                            0.00       0.00       0.00          1
    playlist_name (label_id: 40)                             0.00       0.00       0.00          1
    podcast_descriptor (label_id: 41)                        0.00       0.00       0.00         13
    podcast_name (label_id: 42)                              0.00       0.00       0.00          4
    radio_name (label_id: 43)                               66.67      10.53      18.18         38
    relation (label_id: 44)                                  0.00       0.00       0.00         17
    song_name (label_id: 45)                                 0.00       0.00       0.00         22
    time (label_id: 46)                                     70.27      78.20      74.02        133
    time_zone (label_id: 47)                                 0.00       0.00       0.00          9
    timeofday (label_id: 48)                                 0.00       0.00       0.00         28
    transport_agency (label_id: 49)                          0.00       0.00       0.00          9
    transport_descriptor (label_id: 50)                      0.00       0.00       0.00          0
    transport_name (label_id: 51)                            0.00       0.00       0.00          4
    transport_type (label_id: 52)                           78.38      82.86      80.56         35
    weather_descriptor (label_id: 53)                        0.00       0.00       0.00         17
    O (label_id: 54)                                        92.42      98.80      95.50       5920
    -------------------
    micro avg                                               89.10      89.10      89.10       7199
    macro avg                                               21.86      18.56      18.18       7199
    weighted avg                                            84.42      89.10      86.01       7199
```

## Evaluation
To see how the model performs, we can evaluate the performance of the trained model on a test data file. Here we will reload the model from the `.nemo` file saved during training. By default, the `.nemo` file contains the final checkpoint. We will use the same trainer for testing.

In [ ]:
# extract the path of the best checkpoint from the training, you may update it to any other saved checkpoint file
checkpoint_path = trainer.checkpoint_callback.best_model_path

# load the model from this checkpoint
eval_model = nemo_nlp.models.IntentSlotClassificationModel.load_from_checkpoint(checkpoint_path=checkpoint_path)

In [ ]:
# we will setup testing data reusing the same config (test section)
eval_model.setup_test_data(test_data_config=config.model.test_ds)

# run the evaluation on the test dataset
trainer.test(model=eval_model, ckpt_path=None, verbose=False)

## Inference from Examples
Next step to see how the trained model will classify Intents and Slots for given queries from this domain. To improve the predictions you may need to train the model for more than 5 epochs.


In [ ]:
queries = [
    'set alarm for seven thirty am',
    'lower volume by fifty percent',
    'what is my schedule for tomorrow',
]

pred_intents, pred_slots = eval_model.predict_from_examples(queries, config.model.test_ds)

logging.info('The prediction results of some sample queries with the trained model:')
for query, intent, slots in zip(queries, pred_intents, pred_slots):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intent: {intent}')
    logging.info(f'Predicted Slots: {slots}')

## Training Script

If you have NeMo installed locally (eg. cloned from the Github), you can also train the model with the example script: `examples/nlp/intent_slot_classification/intent_slot_classification.py.`
This script contains an example on how to train, evaluate and perform inference with the IntentSlotClassificationModel.

To run a training script, use:

`cd examples/nlp/intent_slot_classification`

`python intent_slot_classification.py model.data_dir=PATH_TO_DATA_DIR`

By default, this script uses examples/nlp/intent_slot_classification/conf/intent_slot_classification_config.py config file, and you may update all the params inside of this config file or alternatively providing them in the command line.

<a id='multi-label'></a>
# Multi-Label Intent Classification
---

As mentioned above, our multi-label model will be very similar the single intent classification model, with the added functionality of predicting multiple different intents for a single query. For example, the query `show all flights and fares from denver to san francisco` would have intents `atis_airfare` and `atis_flight`. From our list of intents found in `dict.intents.csv`, the model checks whether each individual intent is suitable for the given query.

For this tutorial, we will be using the ATIS (Airline Travel Information System) dataset, converting it to a multi-label data format, and then using the new data to train our model.



## Download the dataset and convert it to the NeMo format

In [ ]:
# download the converter files from github for the purpose of this tutorial
DATA_DIR = './multiatis'
NEMO_DIR = './atis'

!mkdir {DATA_DIR}
!mkdir {NEMO_DIR}


files = [f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.dict.intent.csv', 
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.dict.slots.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.dict.vocab.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.intent.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.pkl', 
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.query.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.slots.csv', 
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.intent.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.pkl',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.query.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.slots.csv']

         
for file in files:
    wget.download(file, DATA_DIR)


# download the converter files from github for the purpose of this tutorial
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/import_datasets.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/assistant_utils.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/convert_datasets.py', NEMO_DIR)

# Get original atis dataset
!python {NEMO_DIR}/import_datasets.py --dataset_name=atis --source_data_dir={DATA_DIR} --target_data_dir={DATA_DIR}/nemo_format
# Script will create new files at {DATA_DIR}/new_format
!mkdir {DATA_DIR}/new_format
!python {NEMO_DIR}/convert_datasets.py --source_data_dir={DATA_DIR}/nemo_format --target_data_dir={DATA_DIR}/new_format

## Data Augmentation (Optional)
---

In scenarios when we don't have many training examples with multiple intent labels, data augmentation can be very useful. This can be done by concatenating utterances together, and adding it to our training data. Some ways of concatenating include adding a period or \"and\" between the two utterances. A script has been provided below to help with augmentation, but it can be changed depending on your use case.

In [ ]:
# download the data augmentation script
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/augment_training_data.py', NEMO_DIR)

The script augment_training_data.py allows for four command line arguments to be passed in: 

source_data_dir: directory that contains the original multi-label data <br>
target_data_dir: directory to store the new data directory <br>
num_mixed: number of new utterances to add to dataset per class pair (utterances with labels 1 and 2) <br>
link_string: string that is in between the two utterances (".", "", "and", "with") <br>

In [ ]:
!python {NEMO_DIR}/augment_training_data.py --source_data_dir={DATA_DIR}/new_format --target_data_dir={DATA_DIR}/augmented_data --num_mixed=10

## Training the Model

In [ ]:
# download the model config file from repository for the purpose of this example
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/conf/multi_label_intent_slot_classification_config.yaml', NEMO_DIR)

# print content of the config file
config_file = f"{NEMO_DIR}/multi_label_intent_slot_classification_config.yaml"
print(config_file)
config = OmegaConf.load(config_file)
print(OmegaConf.to_yaml(config))

In [ ]:
config.model.data_dir = f"{DATA_DIR}/new_format"
config.model.validation_ds.prefix = "dev"
config.model.test_ds.prefix = "dev"
config.model.class_balancing = "weighted_loss"
config.trainer.max_epochs = 5
run_name = "test"

# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
config.trainer.accelerator = None

trainer = pl.Trainer(**config.trainer)
config.exp_manager.exp_dir = os.path.join(DATA_DIR, "output/" + run_name)
config.exp_manager.create_checkpoint_callback = True

exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.MultiLabelIntentSlotClassificationModel(config.model, trainer=trainer)
trainer.fit(model)

## Evaluation

To see how the model performs, we can evaluate the performance of the trained model on a test data file. Here we will reload the model from the `.nemo` file saved during training. By default, the `.nemo` file contains the final checkpoint. We will use the same trainer for testing.

In [ ]:
# specify checkpoint path with .nemo file
checkpoint_path = os.path.join(exp_dir, "checkpoints", "MultiLabelIntentSlot.nemo")

# load the model from this checkpoint
eval_model =  nemo_nlp.models.MultiLabelIntentSlotClassificationModel.restore_from(checkpoint_path)

### Optimizing Threshold

As mentioned above, when classifiying a given query such as `show all flights and fares from denver to san francisco`, our model checks whether each individual intent would be suitable. Before assigning the final labels for a query, the model assigns a probability an intent matches the query. For example, if our `dict.intents.csv` had 5 different intents, then the model could output for a given query \[0.52, 0.38, 0.21, 0.67. 0.80\] where each value represents the probability that query matches that particular intent. 

We need to use these probabilities to generate final label predictions of 0 or 1 for each label. While we can use 0.5 as the probability threshold, it is usually the case that there is a better threshold to use depending on the metric we want to optimize. For this tutorial, we will be finding the threshold that gives us the best micro-F1 score on the validation set. After running the `optimize_threshold` method, the threshold attribute for our model will be updated.

In [ ]:
eval_model.optimize_threshold(config.model.test_ds, 'dev')

In [ ]:
eval_model.threshold

###  Inference from Examples
Similar to the previous example we can run inference to see how the trained model will classify Intents and Slots for given queries from this domain. To improve the predictions you may need to train the model for more than 10 epochs.


In [ ]:
queries = [
    'i would like to find a flight from charlotte to las vegas that makes a stop in st. louis',
    'on april first i need a ticket from tacoma to san jose departing before 7 am',
    'how much is the limousine service in boston',
]

# We use the optimized threshold for predictions
pred_intents, pred_slots, pred_list = eval_model.predict_from_examples(queries, config.model.test_ds)
logging.info('The prediction results of some sample queries with the trained model:')
    
for query, intent, slots in zip(queries, pred_intents, pred_slots):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intents: {intent}')
    logging.info(f'Predicted Slots: {slots}')